# Depth Discrimination Experiment Sequences

In [315]:
import os, json
import pandas as pd
import random 
import copy 
import sqlite3
import sys
import numpy as np 
from itertools import permutations  
import itertools
from itertools import combinations
import operator

In [284]:
def getSubdirTargetInfo(directory):
    """
    Indexes into the json file of each image and recursively extracts image characteristics
    All object (target image) instances are added to obs_bins (main list of all images)
    Args: 
        directory = path to cleaned stimuli folder
    Returns:
        obs_bins = instance of Observation_bins class that has depth_ob (instance) for every target image
    """
    obs = {}
    for folder in os.listdir(directory): #recursively goes through all the folders
        if folder != '.DS_Store':
            for file in os.listdir(directory + '/' + folder):
                filepath = directory + '/' + folder + '/' + file
                if ".ipynb_checkpoints" not in str(filepath):
                    if filepath.endswith(".json"):
                        output_json = json.load(open(filepath)) #loads each data.json file
                        objects = output_json['objects'] 
                        for obj in objects:
                            cp = obj["crossing_point"]
                            cp = cp[0] #indexes to the dict
                            depth = cp['depth']
                            obs[folder] = depth

    return obs

In [379]:
path = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/final_stimuli'

# path = '/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/depth_duration_MTurk/depth_duration_stimuli'
observations = getSubdirTargetInfo(path)

In [380]:
# sort observations by increasing depth (dict value)
sorted_tuples = sorted(observations.items(), key=operator.itemgetter(1))
sorted_dict = {k: v for k, v in sorted_tuples}

# create groups of 4 from low to high depth --> 48 groups total 
grouped_observations = [sorted_tuples[i*4:(i*4)+4] for i in range(48)]
len(grouped_observations)

48

### Create Sequences

In [381]:
# tuples represent the indeces of images within the 4 image group 
# each sublist has 2 combinations with no image repetition 
combs = [[(0,1),(2,3)], [(0,2), (1,3)], [(0,3), (1,2)]]
bins_combinations = [copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs),
                     copy.deepcopy(combs),copy.deepcopy(combs),copy.deepcopy(combs)]
print(len(bins_combinations))

sequences = np.array([[None]*48]*3, dtype=object)
print(len(sequences), sequences[0].shape)


48
3 (48,)


In [382]:
for seq in range(3): # 3 sequences total to completely cover the set of comparisons 
    for i in range(len(bins_combinations)): # 48 bins
        # randomly select a list of combos 
        # the random pick is a list of two sets of image combos within that bin 
        random_picks = random.choice(bins_combinations[i])
        random_idx = bins_combinations[i].index(random_picks)

        # remove the selected image index combo 
        del bins_combinations[i][random_idx]

        # add selected combo to sequence[seq]
        sequences[seq][i]= random_picks
        

In [383]:
sequences[0].shape

(48,)

In [384]:
sequences[0][0]

[(0, 3), (1, 2)]

### Assign Images based on Sequence

#### Create image order rotation sequence too

In [389]:
destination = '/Users/prachi/Documents/depth_duration/depth_discrimination/discrimination_sequences'


In [390]:
sequence_dictionaries = []
rotated_sequence_dictionaries = [] # order of images is flipped
seq_names = ['c0', 'c1', 'c2']
durations = [250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000, 250, 1000,
             250, 1000, 250, 1000, 250, 1000, 250, 1000]
# shuffle durations 
random.shuffle(durations)

for i in range(len(sequences)):
    seq = [] 
    rotated_seq = []
    sequence_name = seq_names[i]
    rotated_seq_name = seq_names[i] + '_rotated' # order rotated
    num = 0 # iterator for bins (48 total)
    dur_num = 0 # iterator that ensures each mask is unique and is the index for durations (list)
    for bin_trials in sequences[i]: # loop through the 48 bins
        for trial in bin_trials: # loop through the 2 sets of image combos ex. [(0,1), (2, 3)]
            img0_index = trial[0]
            img1_index = trial[1]
            dict_trial = {}
            dict_trial["sequence"] = sequence_name
            dict_trial["duration"] = durations[num]
            dict_trial["depth_0"] = grouped_observations[num][img0_index][1]
            dict_trial["depth_1"] = grouped_observations[num][img1_index][1]

            targetimg_0 = grouped_observations[num][img0_index][0]
            targetimg_1 = grouped_observations[num][img1_index][0]
            # this has to be the path on the server
            targetimg_0_path = "depth_discrimination_stimuli/" + targetimg_0 + '/' + targetimg_0 + '-target.png'
            targetimg_1_path = "depth_discrimination_stimuli/" + targetimg_1 + '/' + targetimg_1 + '-target.png'

            dict_trial["image_path_target_0"] = targetimg_0_path
            dict_trial["image_path_target_1"] = targetimg_1_path

            dict_trial["mask_path"] = "masks/mask_" + str(num) + ".jpg"
            dict_trial["fixation_path"] = "fixation.jpg"

            seq.append(dict_trial)
            ###################### Rotated sequence ######################
            rotated_dict_trial = {}
            rotated_dict_trial["sequence"] = rotated_seq_name
            rotated_dict_trial["duration"] = durations[dur_num]
            dur_num += 1
            rotated_dict_trial["depth_0"] = grouped_observations[num][img1_index][1]
            rotated_dict_trial["depth_1"] = grouped_observations[num][img0_index][1]

            r_targetimg_0 = grouped_observations[num][img1_index][0]
            r_targetimg_1 = grouped_observations[num][img0_index][0]
            # this has to be the path on the server
            r_targetimg_0_path = "depth_discrimination_stimuli/" + r_targetimg_1 + '/' + r_targetimg_1 + '-target.png'
            r_targetimg_1_path = "depth_discrimination_stimuli/" + r_targetimg_0 + '/' + r_targetimg_0 + '-target.png'

            rotated_dict_trial["image_path_target_0"] = r_targetimg_1_path
            rotated_dict_trial["image_path_target_1"] = r_targetimg_0_path

            rotated_dict_trial["mask_path"] = "masks/mask_" + str(dur_num) + ".jpg"
            rotated_dict_trial["fixation_path"] = "fixation.jpg"

            rotated_seq.append(rotated_dict_trial)

        num += 1

    # shuffle the order of trials so that trials are not in order of increasing depth 
    # use the same random seed for sequence & it's rotation so trials match in images 
    random.Random(i).shuffle(seq)
    random.Random(i).shuffle(rotated_seq)
    random.Random(i).shuffle(seq)
    random.Random(i).shuffle(rotated_seq)

    # reshuffle durations so that each sequence has a different duration order 
    random.shuffle(durations)
    
    sequence_dictionaries.append(seq)
    rotated_sequence_dictionaries.append(rotated_seq)

    
for sequence in sequence_dictionaries:
    name = sequence[0]["sequence"]
    path = destination + '/' + name + '.json'
    # creates json file for the sequence 
    with open(path, 'w') as f:
        json.dump(sequence , f)  
        
for rotated_sequence in rotated_sequence_dictionaries:
    rotated_name = rotated_sequence[0]["sequence"]
    rotated_path = destination + '/' + rotated_name + '.json'
    # creates json file for the sequence 
    with open(rotated_path, 'w') as f:
        json.dump(rotated_sequence , f)  

### Rotate Sequence by Duration

In [391]:
def load_master_sequence(jsonpath):
    return json.load(open(jsonpath))

def rotate_sequence(previous_seq):
    """
    Rotates each trial's duration assignment based on previous sequence 
    250 --> 1000
    1000 --> 250
    """
    rotated = previous_seq
    for i in range(len(previous_seq)):
        duration = previous_seq[i]['duration']
        if duration == 1000:
            new_duration = 250
        else:
            new_duration = 1000
        rotated[i]['duration'] = new_duration
        
    return rotated

def create_duration_rotations(jsonpath, exit, name):
    """
    Args:
        jsonpath = path to master json created through sequence pipeline
        exit = destination path for new jsons 
        name = i.e. V1 
    
    Creates sequences rotated by duration so that all images in the master sequence are seen at each duration
    (across participants)
        
    """
    master = load_master_sequence(jsonpath)
    
    r = rotate_sequence(master)
    r_path = exit + '/' + name + '_dr.json' # duration rotated sequence
    #creates json file for the sequence 
    with open(r_path, 'w') as f:
        json.dump(r , f)

        
def main_seq_rotations(json_folderpath, exit):
    """
    Create rotated sequence for each sequence in the folder
    """
    for file in os.listdir(json_folderpath):
        name = file.split(".")[0]
        jsonpath = json_folderpath + "/" + file
        try:
            create_duration_rotations(jsonpath, exit, name)
        except:
            print("Failed to create json rotations for: ", file)

In [392]:
main_seq_rotations(destination,destination)

Failed to create json rotations for:  .DS_Store
Failed to create json rotations for:  .ipynb_checkpoints


### Sequences:
Original Sequence: c0 
- c0_dr: duration assigned is rotated
- c0_rotated: order of images if flipped
- c0_rotated_dr: duration assigned is rotated

3 original sequences 
- covers the whole space of 2 trial combinations within the 48 bins 
- 96 trials total, with 2 trials from each bin 


12 Total sequences

In [280]:
count = 0
for js in os.listdir('/Users/prachi/Documents/depth_duration/depth_discrimination/discrimination_sequences'):
    if '.json' in js:
        count += 1

count

24

5 min, 1 dollar
- test time it takes to complete experiment

Consider running experiment bundles
- prefer orthogonal tasks
- counterbalance the order of the experiments